### Import dependencies

In [1]:
import pandas as pd
import numpy as np
from functools import reduce
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

/Users/gfidanli/anaconda3/envs/python3_default/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/gfidanli/anaconda3/envs/python3_default/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/gfidanli/anaconda3/envs/python3_default/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/gfi

### Merge datasets
Add in new data sets by reading in the csv and adding that object into the "dfs" list below

In [2]:
# Read in data
cpi = pd.read_csv("../resources/cpi_final.csv")
gdp = pd.read_csv("../resources/gdp_final.csv")
gdp_pct = pd.read_csv("../resources/gdp_pct_chg_final.csv")
houst = pd.read_csv("../resources/housing_starts_final.csv")
opg = pd.read_csv("../resources/output_gap_final.csv")
rec_dt = pd.read_csv("../resources/recession_dates_final.csv")
unrate = pd.read_csv("../resources/unemployment_rate_final.csv")
fed_funds = pd.read_csv("../resources/fed_funds_final.csv")
yield10_2 = pd.read_csv("../resources/10YT_minus_2YT_final.csv")
fred = pd.read_csv("../resources/FRED_data.csv")

In [3]:
# Combine all data sets into one data frame
dfs = [cpi, gdp, gdp_pct, houst, opg, rec_dt, unrate, fed_funds, yield10_2, fred]
df = reduce(lambda  left,right: pd.merge(left,right,on=['quarter'],how='outer'), dfs)
df.head()

,quarter,avg_consumer_price_index,date_x,gdp,date_y,gdp_pct_change,avg_housing_starts,date_x,output_gap,date_y,...,nat_rate_of_unemp_long_term_PCH,personal_consumption_expenditures,personal_consumption_expenditures_CCA,personal_consumption_expenditures_CCH,personal_consumption_expenditures_CH1,personal_consumption_expenditures_CHG,personal_consumption_expenditures_LOG,personal_consumption_expenditures_PC1,personal_consumption_expenditures_PCA,personal_consumption_expenditures_PCH
0,1947Q1,21.700000,1947-01-01,243.164,NaN,NaN,NaN,NaN,NaN,1947-01-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1947Q2,22.010000,1947-04-01,245.968,1947-04-01,4.7,NaN,NaN,NaN,1947-04-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1947Q3,22.490000,1947-07-01,249.585,1947-07-01,6.0,NaN,NaN,NaN,1947-07-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1947Q4,23.126667,1947-10-01,259.745,1947-10-01,17.3,NaN,NaN,NaN,1947-10-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1948Q1,23.616667,1948-01-01,265.742,1948-01-01,9.6,NaN,NaN,NaN,1948-01-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Drop date columns
df = df.drop(columns=['date_x','date_y'])

In [5]:
# Sort data frame by quarter
df = df.sort_values(by=['quarter'])

In [6]:
# Check dataset before removing nulls
df.tail()

,quarter,avg_consumer_price_index,gdp,gdp_pct_change,avg_housing_starts,output_gap,target,avg_unemployment_rate,fed_funds_avg_rate,fed_funds_percent_change_prev_quarter,...,nat_rate_of_unemp_long_term_PCH,personal_consumption_expenditures,personal_consumption_expenditures_CCA,personal_consumption_expenditures_CCH,personal_consumption_expenditures_CH1,personal_consumption_expenditures_CHG,personal_consumption_expenditures_LOG,personal_consumption_expenditures_PC1,personal_consumption_expenditures_PCA,personal_consumption_expenditures_PCH
287,2018Q4,252.759000,20897.804,2.9,1185.000000,0.592021,0.0,3.566667,2.217097,0.152641,...,-0.1,14211.920,2.7,0.7,625.653,97.361,9.6,4.6,2.8,0.7
288,2019Q1,253.311333,21098.827,3.9,1213.000000,0.848147,0.0,4.133333,2.401311,0.083088,...,-0.1,14266.250,1.5,0.4,537.893,54.330,9.6,3.9,1.5,0.4
289,2019Q2,255.139333,21340.267,4.7,1255.666667,0.828815,NaN,3.500000,2.397813,-0.001457,...,-0.1,14511.176,6.8,1.7,571.348,244.926,9.6,4.1,7.0,1.7
290,2019Q3,256.273000,NaN,NaN,1282.000000,NaN,NaN,3.700000,2.197813,-0.083409,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
311,2019Q4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.845625,-0.160245,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Drop rows with missing values
df = df.dropna()

In [8]:
# Check dataset after removing nulls
df.tail()

,quarter,avg_consumer_price_index,gdp,gdp_pct_change,avg_housing_starts,output_gap,target,avg_unemployment_rate,fed_funds_avg_rate,fed_funds_percent_change_prev_quarter,...,nat_rate_of_unemp_long_term_PCH,personal_consumption_expenditures,personal_consumption_expenditures_CCA,personal_consumption_expenditures_CCH,personal_consumption_expenditures_CH1,personal_consumption_expenditures_CHG,personal_consumption_expenditures_LOG,personal_consumption_expenditures_PC1,personal_consumption_expenditures_PCA,personal_consumption_expenditures_PCH
284,2018Q1,249.250333,20163.159,5.0,1320.666667,0.202456,0.0,4.333333,1.448966,0.204683,...,-0.1,13728.357,4.2,1.0,623.938,142.090,9.5,4.8,4.2,1.0
285,2018Q2,250.578667,20510.177,7.1,1259.666667,0.589182,0.0,3.833333,1.727176,0.192007,...,-0.1,13939.828,6.1,1.5,727.327,211.471,9.5,5.5,6.3,1.5
286,2018Q3,251.828667,20749.752,4.8,1233.000000,0.821959,0.0,3.866667,1.923492,0.113663,...,-0.1,14114.559,5.0,1.2,769.506,174.731,9.6,5.8,5.1,1.3
287,2018Q4,252.759000,20897.804,2.9,1185.000000,0.592021,0.0,3.566667,2.217097,0.152641,...,-0.1,14211.920,2.7,0.7,625.653,97.361,9.6,4.6,2.8,0.7
288,2019Q1,253.311333,21098.827,3.9,1213.000000,0.848147,0.0,4.133333,2.401311,0.083088,...,-0.1,14266.250,1.5,0.4,537.893,54.330,9.6,3.9,1.5,0.4


In [9]:
# Set index to quarter
df = df.set_index('quarter')

In [10]:
# Rename target column
df = df.rename(columns={'target':'recession_actual'})
df.head()

,avg_consumer_price_index,gdp,gdp_pct_change,avg_housing_starts,output_gap,recession_actual,avg_unemployment_rate,fed_funds_avg_rate,fed_funds_percent_change_prev_quarter,fed_funds_st_dev_rate,...,nat_rate_of_unemp_long_term_PCH,personal_consumption_expenditures,personal_consumption_expenditures_CCA,personal_consumption_expenditures_CCH,personal_consumption_expenditures_CH1,personal_consumption_expenditures_CHG,personal_consumption_expenditures_LOG,personal_consumption_expenditures_PC1,personal_consumption_expenditures_PCA,personal_consumption_expenditures_PCH
quarter,,,,,,,,,,,,,,,,,,,,,
1976Q3,57.300000,1886.558,7.6,1557.000000,-2.199151,0.0,7.600000,5.283478,0.016956,0.100618,...,0.1,1158.806,10.2,2.6,111.614,29.266,7.1,10.7,10.8,2.6
1976Q4,58.133333,1934.273,10.5,1691.333333,-2.246705,0.0,7.333333,4.874239,-0.077456,0.211941,...,0.1,1192.408,11.4,2.9,116.185,33.602,7.1,10.8,12.1,2.9
1977Q1,59.200000,1988.648,11.7,1844.333333,-1.877175,0.0,8.233333,4.660667,-0.043817,0.148254,...,0.1,1228.212,11.8,3.0,118.304,35.804,7.1,10.7,12.6,3.0
1977Q2,60.233333,2055.909,14.2,1918.666667,-0.776696,0.0,6.933333,5.157473,0.106595,0.332835,...,0.1,1255.980,8.9,2.2,126.440,27.768,7.1,11.2,9.4,2.3
1977Q3,61.066667,2118.473,12.7,2009.000000,0.186001,0.0,6.800000,5.816413,0.127764,0.344309,...,0.0,1286.905,9.7,2.4,128.099,30.925,7.2,11.1,10.2,2.5


### Shift data with sliding window technique

In [11]:
df['recession_1q_out'] = df['recession_actual'].shift(-1)
df['recession_2q_out'] = df['recession_actual'].shift(-2)
df['recession_4q_out'] = df['recession_actual'].shift(-4)

In [12]:
# Create three datasets -- 1 for each model (recession 1Qtr out, 2Qtrs out, 4Qtrs out)
df_q1 = df.drop(columns=['recession_2q_out','recession_4q_out','recession_actual'])
df_q2 = df.drop(columns=['recession_4q_out','recession_1q_out','recession_actual'])
df_q4 = df.drop(columns=['recession_1q_out','recession_2q_out','recession_actual'])

In [13]:
# Delete missing values
df_q1 = df_q1.dropna()
df_q2 = df_q2.dropna()
df_q4 = df_q4.dropna()
df_q4.tail()

,avg_consumer_price_index,gdp,gdp_pct_change,avg_housing_starts,output_gap,avg_unemployment_rate,fed_funds_avg_rate,fed_funds_percent_change_prev_quarter,fed_funds_st_dev_rate,10YT_minus_2YT_avg,...,personal_consumption_expenditures,personal_consumption_expenditures_CCA,personal_consumption_expenditures_CCH,personal_consumption_expenditures_CH1,personal_consumption_expenditures_CHG,personal_consumption_expenditures_LOG,personal_consumption_expenditures_PC1,personal_consumption_expenditures_PCA,personal_consumption_expenditures_PCH,recession_4q_out
quarter,,,,,,,,,,,,,,,,,,,,,
2017Q1,243.830000,19190.431,4.2,1230.666667,-0.861917,4.866667,0.698889,0.559490,0.098830,1.203333,...,13104.419,4.4,1.1,580.895,144.652,9.5,4.6,4.5,1.1,0.0
2017Q2,244.065000,19356.649,3.5,1169.333333,-0.752038,4.233333,0.947363,0.355527,0.098588,0.970000,...,13212.501,3.3,0.8,524.240,108.082,9.5,4.1,3.3,0.8,0.0
2017Q3,245.368333,19611.704,5.4,1175.333333,-0.396155,4.400000,1.153696,0.217797,0.024029,0.880000,...,13345.053,4.0,1.0,522.673,132.552,9.5,4.1,4.1,1.0,0.0
2017Q4,247.273333,19918.910,6.4,1259.666667,0.033653,3.900000,1.202778,0.042543,0.099120,0.673333,...,13586.267,7.2,1.8,626.500,241.214,9.5,4.8,7.4,1.8,0.0
2018Q1,249.250333,20163.159,5.0,1320.666667,0.202456,4.333333,1.448966,0.204683,0.083902,0.596667,...,13728.357,4.2,1.0,623.938,142.090,9.5,4.8,4.2,1.0,0.0


In [14]:
# Define y variables
y1 = df_q1['recession_1q_out']
y2 = df_q2['recession_2q_out']
y3 = df_q4['recession_4q_out']

In [15]:
# Drop target
df_q1 = df_q1.drop(columns=['recession_1q_out'])
df_q2 = df_q2.drop(columns=['recession_2q_out'])
df_q4 = df_q4.drop(columns=['recession_4q_out'])

In [16]:
# Define X
X_q1 = df_q1
X_q2 = df_q2
X_q4 = df_q4

### Split and scale data

In [17]:
# Split data into training and testing
X1_train, X1_test, y1_train, y1_test=train_test_split(X_q1,y1, train_size=0.8, random_state=42, stratify=y1)
X2_train, X2_test, y2_train, y2_test=train_test_split(X_q2,y2, train_size=0.8, random_state=42, stratify=y2)
X3_train, X3_test, y3_train, y3_test=train_test_split(X_q4,y3, train_size=0.8, random_state=42, stratify=y3)

In [18]:
# Remove shuffle to see if model performs better
# Split data into training and testing
X1_train, X1_test, y1_train, y1_test=train_test_split(X_q1,y1, train_size=0.8, random_state=42, shuffle=False)
X2_train, X2_test, y2_train, y2_test=train_test_split(X_q2,y2, train_size=0.8, random_state=42, shuffle=False)
X3_train, X3_test, y3_train, y3_test=train_test_split(X_q4,y3, train_size=0.8, random_state=42, shuffle=False)

In [19]:
# Create scaler object
X1_scaler = StandardScaler().fit(X1_train)
X2_scaler = StandardScaler().fit(X2_train)
X3_scaler = StandardScaler().fit(X3_train)

In [20]:
# Scale training data
X1_train_scaled = X1_scaler.transform(X1_train)
X2_train_scaled = X2_scaler.transform(X2_train)
X3_train_scaled = X3_scaler.transform(X3_train)

# Scale testing data
X1_test_scaled = X1_scaler.transform(X1_test)
X2_test_scaled = X2_scaler.transform(X2_test)
X3_test_scaled = X3_scaler.transform(X3_test)

### Reshape data using np.reshape

In [21]:
# Method to reshape data
def reshape_data(obj):
    reshaped_obj = np.reshape(obj, (obj.shape[0], obj.shape[1], 1))
    return reshaped_obj

In [22]:
# Reshape training data
reshaped_X1_train_scaled = reshape_data(X1_train_scaled)
reshaped_X2_train_scaled = reshape_data(X2_train_scaled)
reshaped_X3_train_scaled = reshape_data(X3_train_scaled)

In [23]:
# Reshape testing data
reshaped_X1_test_scaled = reshape_data(X1_test_scaled)
reshaped_X2_test_scaled = reshape_data(X2_test_scaled)
reshaped_X3_test_scaled = reshape_data(X3_test_scaled)

## Build Model

In [24]:
# Initialize model
model = Sequential()

In [25]:
# Add layers
model.add(LSTM(128, input_shape=(reshaped_X1_train_scaled.shape[1],1), return_sequences=True))
model.add(Dropout(0.4))
model.add(BatchNormalization())  # Normalize activation outputs

model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(LSTM(128))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(2, activation='softmax'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [26]:
# Compile model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])

**Train and predict on X1-Y1 data (recession 1 quarter out)**

In [27]:
# Fit the model to the training data
# Shuffle True/False to randomize the training data rows being fed into the model
model.fit(reshaped_X1_train_scaled, y1_train, epochs=100, shuffle=False, verbose=2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
136/136 - 14s - loss: 0.9407 - acc: 0.5294
Epoch 2/100
136/136 - 6s - loss: 0.7633 - acc: 0.6029
Epoch 3/100
136/136 - 6s - loss: 0.6536 - acc: 0.6691
Epoch 4/100
136/136 - 7s - loss: 0.6990 - acc: 0.6618
Epoch 5/100
136/136 - 7s - loss: 0.4939 - acc: 0.7794
Epoch 6/100
136/136 - 6s - loss: 0.6377 - acc: 0.6618
Epoch 7/100
136/136 - 6s - loss: 0.6865 - acc: 0.7206
Epoch 8/100
136/136 - 6s - loss: 0.6004 - acc: 0.7353
Epoch 9/100
136/136 - 6s - loss: 0.5020 - acc: 0.7500
Epoch 10/100
136/136 - 6s - loss: 0.4965 - acc: 0.7794
Epoch 11/100
136/136 - 6s - loss: 0.4623 - acc: 0.8015
Epoch 12/100
136/136 - 6s - loss: 0.3618 - acc: 0.8162
Epoch 13/100
136/136 - 7s - loss: 0.4024 - acc: 0.7941
Epoch 14/100
136/136 - 6s - loss: 0.5684 - acc: 0.7721
Epoch 15/100
136/136 - 6s - loss: 0.3782 - acc: 0.8456
Epoch 16/100
136/136 - 6s - loss: 0.5032 - acc: 0.7794
Epoch 17/100
136/136 - 6s - loss: 

KeyboardInterrupt: 

In [76]:
# Validate model using test data
model_loss1, model_accuracy1 = model.evaluate(reshaped_X1_test_scaled, y1_test, verbose=2)

21/21 - 0s - loss: 1.0643 - acc: 0.9048


In [77]:
# Make predictions using test data
predictions1 = model.predict_classes(reshaped_X1_test_scaled)
predictions1

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [78]:
X1_test

,avg_consumer_price_index,gdp,gdp_pct_change,avg_housing_starts,output_gap,avg_unemployment_rate,fed_funds_avg_rate,fed_funds_percent_change_prev_quarter,fed_funds_st_dev_rate,10YT_minus_2YT_avg,...,personal_consumption_expenditures_PCH,hh_debt_service_pmts_as_pct_of_disp_personal_income,hh_debt_service_pmts_as_pct_of_disp_personal_income_CCA,hh_debt_service_pmts_as_pct_of_disp_personal_income_CCH,hh_debt_service_pmts_as_pct_of_disp_personal_income_CH1,hh_debt_service_pmts_as_pct_of_disp_personal_income_CHG,hh_debt_service_pmts_as_pct_of_disp_personal_income_LOG,hh_debt_service_pmts_as_pct_of_disp_personal_income_PC1,hh_debt_service_pmts_as_pct_of_disp_personal_income_PCA,hh_debt_service_pmts_as_pct_of_disp_personal_income_PCH
quarter,,,,,,,,,,,,,,,,,,,,,
2007Q3,207.939000,14535.003,4.3,1289.000000,0.319771,4.666667,5.074348,-0.033902,0.226412,0.350000,...,1.1,13.076776,3.7,0.9,0.288954,0.120118,2.6,2.3,3.8,0.9
2007Q4,210.489667,14681.501,4.1,1166.000000,0.461882,4.566667,4.495652,-0.114043,0.279384,0.783333,...,1.3,13.215388,4.2,1.1,0.345694,0.138612,2.6,2.7,4.3,1.1
2008Q1,212.769667,14651.039,-0.8,1064.000000,-0.575539,5.266667,3.181429,-0.292332,0.658310,1.640000,...,0.6,13.135629,-2.4,-0.6,0.254222,-0.079759,2.6,2.0,-2.4,-0.6
2008Q2,215.537667,14805.611,4.3,1010.666667,-0.519069,5.233333,2.084725,-0.344720,0.162326,1.463333,...,1.2,12.792065,-10.6,-2.7,-0.164593,-0.343564,2.5,-1.3,-10.1,-2.6
2010Q2,217.297333,14926.098,5.7,602.000000,-4.312374,9.466667,0.191978,0.433861,0.017011,2.620000,...,0.9,11.350135,-11.5,-2.9,-1.020370,-0.331686,2.4,-8.2,-10.9,-2.8
2010Q3,217.934333,15079.917,4.2,579.666667,-3.851194,9.466667,0.188804,-0.016531,0.011372,2.246667,...,0.9,11.144193,-7.3,-1.8,-1.091769,-0.205942,2.4,-8.9,-7.1,-1.8
2010Q4,219.699000,15240.843,4.3,542.333333,-3.622871,9.133333,0.189565,0.004030,0.013580,2.380000,...,1.3,10.980496,-5.9,-1.5,-0.982179,-0.163697,2.4,-8.2,-5.7,-1.5
2011Q1,222.043667,15285.828,1.2,582.333333,-4.144935,9.500000,0.154556,-0.184684,0.016904,2.766667,...,1.3,10.773656,-7.6,-1.9,-0.908165,-0.206840,2.4,-7.8,-7.3,-1.9
2011Q4,227.047333,15796.460,5.4,671.666667,-3.332909,8.333333,0.074348,-0.110533,0.008555,1.783333,...,0.5,10.450274,-4.8,-1.2,-0.530222,-0.126926,2.3,-4.8,-4.7,-1.2


**Train and predict on X2-Y2 data (recession 2 quarters out)**

In [75]:
# Fit the model to the training data
model.fit(reshaped_X2_train_scaled, y2_train, epochs=100, shuffle=True, verbose=2)

Epoch 1/100
83/83 - 5s - loss: 0.6904 - acc: 0.9277
Epoch 2/100


KeyboardInterrupt: 

In [50]:
# Validate model using test data
model_loss2, model_accuracy2 = model.evaluate(reshaped_X2_test_scaled, y2_test, verbose=2)

21/21 - 0s - loss: 0.5653 - acc: 0.9048


In [51]:
# Make predictions using test data
predictions2 = model.predict_classes(reshaped_X2_test_scaled)
predictions2

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

**Train and predict on X3-Y3 data (recession 4 quarters out)**

In [52]:
# Fit the model to the training data
model.fit(reshaped_X3_train_scaled, y3_train, epochs=100, shuffle=True, verbose=2)

Epoch 1/100
83/83 - 5s - loss: 0.7856 - acc: 0.8916
Epoch 2/100
83/83 - 5s - loss: 0.5266 - acc: 0.9398
Epoch 3/100
83/83 - 5s - loss: 0.3128 - acc: 0.9157
Epoch 4/100
83/83 - 4s - loss: 0.3255 - acc: 0.8916
Epoch 5/100
83/83 - 4s - loss: 0.2946 - acc: 0.9036
Epoch 6/100
83/83 - 5s - loss: 0.1655 - acc: 0.9036
Epoch 7/100
83/83 - 5s - loss: 0.2359 - acc: 0.9157
Epoch 8/100
83/83 - 5s - loss: 0.1651 - acc: 0.9277
Epoch 9/100
83/83 - 6s - loss: 0.1412 - acc: 0.9639
Epoch 10/100
83/83 - 5s - loss: 0.1275 - acc: 0.9518
Epoch 11/100
83/83 - 5s - loss: 0.1329 - acc: 0.9518
Epoch 12/100
83/83 - 4s - loss: 0.1595 - acc: 0.9036
Epoch 13/100
83/83 - 5s - loss: 0.1147 - acc: 0.9518
Epoch 14/100
83/83 - 5s - loss: 0.1514 - acc: 0.9157
Epoch 15/100
83/83 - 5s - loss: 0.1236 - acc: 0.9036
Epoch 16/100
83/83 - 6s - loss: 0.0892 - acc: 0.9880
Epoch 17/100
83/83 - 5s - loss: 0.0859 - acc: 0.9759
Epoch 18/100
83/83 - 5s - loss: 0.1016 - acc: 0.9518
Epoch 19/100
83/83 - 5s - loss: 0.0670 - acc: 0.9759
Ep

In [ ]:
# Validate model using test data
model_loss3, model_accuracy3 = model.evaluate(reshaped_X3_test_scaled, y3_test, verbose=2)

In [56]:
# Make predictions using test data
predictions3 = model.predict_classes(reshaped_X3_test_scaled)
predictions3

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])

In [58]:
X3_test

,avg_consumer_price_index,gdp,gdp_pct_change,avg_housing_starts,output_gap,avg_unemployment_rate,fed_funds_avg_rate,fed_funds_percent_change_prev_quarter,fed_funds_st_dev_rate,10YT_minus_2YT_avg,...,personal_consumption_expenditures_PCH,hh_debt_service_pmts_as_pct_of_disp_personal_income,hh_debt_service_pmts_as_pct_of_disp_personal_income_CCA,hh_debt_service_pmts_as_pct_of_disp_personal_income_CCH,hh_debt_service_pmts_as_pct_of_disp_personal_income_CH1,hh_debt_service_pmts_as_pct_of_disp_personal_income_CHG,hh_debt_service_pmts_as_pct_of_disp_personal_income_LOG,hh_debt_service_pmts_as_pct_of_disp_personal_income_PC1,hh_debt_service_pmts_as_pct_of_disp_personal_income_PCA,hh_debt_service_pmts_as_pct_of_disp_personal_income_PCH
quarter,,,,,,,,,,,,,,,,,,,,,
2014Q4,236.888333,17849.912,2.9,1049.333333,-1.428640,5.466667,0.101304,0.136585,0.017926,1.740000,...,1.1,9.855932,-0.7,-0.2,-0.220200,-0.017596,2.3,-2.2,-0.7,-0.2
2000Q4,174.233333,10439.025,4.7,1544.000000,1.137224,3.666667,6.474891,-0.006836,0.203554,-0.146667,...,1.4,12.063345,7.6,1.9,0.518005,0.226720,2.5,4.5,7.9,1.9
1990Q2,129.300000,5960.028,6.1,1212.333333,-0.633182,5.300000,8.239341,-0.001050,0.128433,0.106667,...,1.2,11.600660,-1.7,-0.4,-0.130832,-0.048490,2.5,-1.1,-1.7,-0.4
1986Q2,109.033333,4545.340,3.4,1878.000000,-1.440078,7.100000,6.918462,-0.116065,0.174145,0.620000,...,1.0,11.812105,2.5,0.6,0.501361,0.072418,2.5,4.4,2.5,0.6
2010Q4,219.699000,15240.843,4.3,542.333333,-3.622871,9.133333,0.189565,0.004030,0.013580,2.380000,...,1.3,10.980496,-5.9,-1.5,-0.982179,-0.163697,2.4,-8.2,-5.7,-1.5
1982Q1,94.600000,3274.302,-0.8,880.000000,-5.211997,9.500000,14.207444,0.045559,1.070977,-0.233333,...,2.0,10.447258,2.6,0.7,0.159742,0.067933,2.3,1.6,2.6,0.7
1994Q4,149.766667,7455.288,7.0,1472.000000,-0.608550,5.266667,5.167935,0.152159,0.455490,0.680000,...,1.6,10.699630,4.2,1.1,0.331112,0.112672,2.4,3.2,4.3,1.1
1982Q2,95.966667,3331.972,7.2,948.000000,-5.555580,9.366667,14.514176,0.021589,0.751461,-0.220000,...,1.3,10.504029,2.2,0.5,0.087151,0.056771,2.4,0.8,2.2,0.5
1988Q4,120.300000,5399.509,9.1,1551.333333,-0.234600,5.066667,8.468478,0.060750,0.299090,0.253333,...,2.2,11.569183,-4.2,-1.0,-0.237263,-0.122007,2.4,-2.0,-4.1,-1.0
